# Jupyter Notebookでチャットボット
前のセクションで構築したSeq2Seqのモデルを使い、Jupyter Notebook上で賢治botを構築します。  

## 文字の読み込み
使用する文字を読み込みます。  
また、使用できない文字が含まれているかどうか、判定する関数を設定します。

In [ ]:
import pickle

with open('kana_chars.pickle', mode='rb') as f:
    chars_list = pickle.load(f)
print(chars_list)

def is_invalid(message):
    is_invalid =False
    for char in message:
        if char not in chars_list:
            is_invalid = True
    return is_invalid

## 文章のベクトル化
文章をone-hot表現に変換する関数を設定します。  

In [ ]:
import numpy as np

# インデックスと文字で辞書を作成
char_indices = {}
for i, char in enumerate(chars_list):
    char_indices[char] = i
indices_char = {}
for i, char in enumerate(chars_list):
    indices_char[i] = char
    
n_char = len(chars_list)
max_length_x = 128

# 文章をone-hot表現に変換する関数
def sentence_to_vector(sentence):
    vector = np.zeros((1, max_length_x, n_char), dtype=np.bool)
    for j, char in enumerate(sentence):
        vector[0][j][char_indices[char]] = 1
    return vector

## 返答作成用の関数
encoderのモデル、およびdecoderのモデルを読み込み、返答を作成するための関数を設定します。  
関数のコードは前のセクションで扱ったものと同じですが、引数を文字列で受け取る点が異なります。

In [ ]:
from keras.models import load_model

encoder_model = load_model('encoder_model.h5')
decoder_model = load_model('decoder_model.h5')

def respond(message, beta=5):
    vec = sentence_to_vector(message)  # 文字列をone-hot表現に変換
    state_value = encoder_model.predict(vec)
    y_decoder = np.zeros((1, 1, n_char))  # decoderの出力を格納する配列
    y_decoder[0][0][char_indices['\t']] = 1  # decoderの最初の入力はタブ。one-hot表現にする。

    respond_sentence = ""  # 返答の文字列
    while True:
        y, h = decoder_model.predict([y_decoder, state_value])
        p_power = y[0][0] ** beta  # 確率分布の調整
        next_index = np.random.choice(len(p_power), p=p_power/np.sum(p_power)) 
        next_char = indices_char[next_index]  # 次の文字
        
        if (next_char == "\n" or len(respond_sentence) >= max_length_x):
            break  # 次の文字が改行のとき、もしくは最大文字数を超えたときは終了
            
        respond_sentence += next_char
        y_decoder = np.zeros((1, 1, n_char))  # 次の時刻の入力
        y_decoder[0][0][next_index] = 1

        state_value = h  # 次の時刻の状態

    return respond_sentence

## チャットボットの構築
resopond関数を使って、チャットボットのシステムを構築します。  
ひらがな、カタカナと一部の記号以外の文字が入力された場合は、注意を表示するようにします。

In [ ]:
bot_name = "賢治bot"
your_name = input("おなまえをおしえてください。:")
print()

print(bot_name + ": " + "こんにちは、" + your_name + "さん。")
message = ""
while message != "さようなら。":
    
    while True:
        message = input(your_name + ": ")
        if not is_invalid(message):
            break
        else:
            print(bot_name + ": ひらがなか、カタカナをつかってください。")
            
    response = respond(message)
    print(bot_name + ": " + response)

的を得ていない回答も多数ありますが、人間の方が想像力で補うケースも多々あるように思えます。  
言葉を覚えたての幼児と、会話をしている感覚に近いかもしれません。  

賢治botはビジネスの場では使いづらそうですが、雑談やコミュニケーション目的での用途はありそうです。  
今回は宮沢賢治のいくつかの小説をコーパスに使いましたが、それぞれの用途に合ったコーパスで学習を行えば、より実務的なチャットボットが出来上がるでしょう。

## 課題
2人がチャットボット、1人が人間の、3者で会話をするシステムを構築してみましょう。